# Import

In [9]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

# Reading Session Keys

In [10]:
session_key_list = pd.read_csv('session_keys.csv')

In [11]:
for i in session_key_list['session_key']:
    response = urlopen(f'https://api.openf1.org/v1/drivers?session_key={i}')
    data = json.loads(response.read().decode('utf-8'))
    if i == 7953:
        df_drivers = pd.DataFrame(data)
    else:
        hold = pd.DataFrame(data)
        df_drivers = pd.concat([df_drivers,hold])
        time.sleep(0.5)
    print(f'Session {i} extracted.')
print('Done')

Session 7953 extracted.
Session 7779 extracted.
Session 7787 extracted.
Session 9070 extracted.
Session 9078 extracted.
Session 9094 extracted.
Session 9102 extracted.
Session 9110 extracted.
Session 9118 extracted.
Session 9126 extracted.
Session 9133 extracted.
Session 9141 extracted.
Session 9149 extracted.
Session 9157 extracted.
Session 9165 extracted.
Session 9173 extracted.
Session 9221 extracted.
Session 9213 extracted.
Session 9181 extracted.
Session 9205 extracted.
Session 9189 extracted.
Session 9197 extracted.
Session 9472 extracted.
Session 9480 extracted.
Session 9488 extracted.
Session 9496 extracted.
Session 9673 extracted.
Session 9507 extracted.
Session 9515 extracted.
Session 9523 extracted.
Session 9531 extracted.
Session 9539 extracted.
Session 9550 extracted.
Session 9558 extracted.
Session 9566 extracted.
Session 9574 extracted.
Session 9582 extracted.
Session 9590 extracted.
Session 9598 extracted.
Session 9606 extracted.
Session 9617 extracted.
Session 9625 ext

In [12]:
df_drivers

,meeting_key,session_key,driver_number,broadcast_name,full_name,name_acronym,team_name,team_colour,first_name,last_name,headshot_url,country_code
0,1141,7953,1,M VERSTAPPEN,Max VERSTAPPEN,VER,Red Bull Racing,3671C6,Max,Verstappen,https://www.formula1.com/content/dam/fom-websi...,NED
1,1141,7953,2,L SARGEANT,Logan SARGEANT,SAR,Williams,37BEDD,Logan,Sargeant,https://www.formula1.com/content/dam/fom-websi...,USA
2,1141,7953,4,L NORRIS,Lando NORRIS,NOR,McLaren,F58020,Lando,Norris,https://www.formula1.com/content/dam/fom-websi...,GBR
3,1141,7953,10,P GASLY,Pierre GASLY,GAS,Alpine,2293D1,Pierre,Gasly,https://www.formula1.com/content/dam/fom-websi...,FRA
4,1141,7953,11,S PEREZ,Sergio PEREZ,PER,Red Bull Racing,3671C6,Sergio,Perez,https://www.formula1.com/content/dam/fom-websi...,MEX
...,...,...,...,...,...,...,...,...,...,...,...,...
15,1276,9839,44,L HAMILTON,Lewis HAMILTON,HAM,Ferrari,ED1131,Lewis,Hamilton,https://media.formula1.com/d_driver_fallback_i...,None
16,1276,9839,55,C SAINZ,Carlos SAINZ,SAI,Williams,1868DB,Carlos,Sainz,https://media.formula1.com/d_driver_fallback_i...,None
17,1276,9839,63,G RUSSELL,George RUSSELL,RUS,Mercedes,00D7B6,George,Russell,https://media.formula1.com/d_driver_fallback_i...,None
18,1276,9839,81,O PIASTRI,Oscar PIASTRI,PIA,McLaren,F47600,Oscar,Piastri,https://media.formula1.com/d_driver_fallback_i...,None


# Upload raw_sessions to PSQL

In [13]:
config = dotenv_values()

pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [14]:
df_drivers.columns

Index(['meeting_key', 'session_key', 'driver_number', 'broadcast_name',
       'full_name', 'name_acronym', 'team_name', 'team_colour', 'first_name',
       'last_name', 'headshot_url', 'country_code'],
      dtype='object')

In [15]:
# defining data types for the DB
dtype_dict = {
    'meeting_key': types.Integer,
    'session_key': types.Integer,
    'driver_number': types.Integer,
    'broadcast_name': types.String,
    'full_name': types.String,
    'name_acronym': types.String,
    'team_name': types.String,
    'team_colour': types.String,        # hex color, keep as string
    'first_name': types.String,
    'last_name': types.String,
    'headshot_url': types.String,
    'country_code': types.String
}

In [16]:
df_drivers.to_sql(name = 'raw_drivers', 
                       con = engine, 
                       schema = pg_schema, 
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

377